In [ ]:
from mymath import *
from potentials import *
import torch
import plotly.graph_objects as go
import numpy as np
from ipywidgets import widgets

In [ ]:
# # Visualization of cubic spline

# x = torch.linspace(-2.5, 2.5, 500, requires_grad=True)
# y = spline3(x)
# E = torch.sum(y)
# deriv1 = torch.autograd.grad(E, x, create_graph=True)[0]
# E = torch.sum(deriv1)
# deriv2 = torch.autograd.grad(E, x, create_graph=True)[0]

# fig = go.Figure(data=[
#     go.Scatter(x=x.detach().numpy(), y=y.detach().numpy(), name="y(x)"),
#     go.Scatter(x=x.detach().numpy(), y=deriv1.detach().numpy(), name="y'(x)"),
#     go.Scatter(x=x.detach().numpy(), y=deriv2.detach().numpy(), name="y''(x)")
#     ], layout=go.Layout(width=400, height=400))
# fig.update_yaxes(exponentformat = 'E')
# fig.show()

In [ ]:
e0 = torch.tensor([0., 0.])
e1 = torch.tensor([1., 0.])
p = torch.tensor([0.5, 0.1])

eps = 5e-1
r = 1
alpha = 2
n_samples = 1000

uv = torch.linspace(0, 1, n_samples, requires_grad=True)
integrand = point_edge_potential_exact_pointwise(p, e0, e1, eps, r, alpha, uv)
E = torch.sum(integrand)
deriv = torch.autograd.grad(E, uv, create_graph=True)[0]

sample_slider = widgets.IntSlider(
    value=1000,
    min=5,
    max=1e6,
    step=1,
    description='n samples:',
    continuous_update=False,
    readout_format='.0f'
)
r_slider = widgets.FloatSlider(
    value=1,
    min=1,
    max=5,
    step=1e-1,
    description='barrier p:',
    continuous_update=False,
    readout_format='.2e'
)
alpha_slider = widgets.FloatSlider(
    value=2,
    min=0,
    max=10,
    step=1e-2,
    description='alpha:',
    continuous_update=False,
    readout_format='.2e'
)
eps_slider = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=2,
    step=1e-3,
    description='eps:',
    continuous_update=False,
    readout_format='.3e'
)
X_slider = widgets.FloatSlider(
    value=0.5,
    min=-0.2,
    max=1.2,
    step=1e-3,
    description='X:',
    continuous_update=False,
    readout_format='.3e'
)
Y_slider = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=0.2,
    step=1e-3,
    description='Y:',
    continuous_update=False,
    readout_format='.3e'
)

sliders = [r_slider,alpha_slider,eps_slider,X_slider,Y_slider,sample_slider]
container = widgets.VBox([
    widgets.HBox([r_slider,alpha_slider,eps_slider]),
    widgets.HBox([X_slider,Y_slider,sample_slider])])

trace1 = go.Scatter(x=uv.detach().numpy(), y=integrand.detach().numpy(), name="Integrand")
trace2 = go.Scatter(x=uv.detach().numpy(), y=deriv.detach().numpy(), name="Derivative")
g = go.FigureWidget(data=[trace1, trace2],
                    layout=go.Layout(width=800, height=600))

def response(change):
    uv = torch.linspace(0, 1, sample_slider.value, requires_grad=True)
    integrand = point_edge_potential_exact_pointwise(torch.tensor([X_slider.value, Y_slider.value]).view(1,-1),
                                                       e0, e1, eps_slider.value, r_slider.value, alpha_slider.value, uv)
    E = torch.sum(integrand)
    deriv = torch.autograd.grad(E, uv, create_graph=True)[0]
    
    with g.batch_update():
        g.data[0].y = integrand.detach().numpy()
        g.data[1].y = deriv.detach().numpy()
        g.layout.xaxis.title = 'UV'
        g.layout.yaxis.title = 'Integrand'

for s in sliders:
    s.observe(response, names="value")

widgets.VBox([container,
              g])

In [ ]:
import meshio

m = meshio.read('smooth-contact-curve.obj')
v = m.points[:,[0,2]]
v = np.vstack((v, v[0,:]))
v[:,1] *= -1

p = v[50,:] + [0, 1e-1]
fig = go.Figure(data = [go.Scatter(x=v[:,0], y=v[:,1]),
                        go.Scatter(x=[p[0]], y=[p[1]])])
fig.update_xaxes(
    scaleanchor="y",
    scaleratio=1,
  )
fig.show()

lengths = np.linalg.norm(v[1:,:] - v[:-1,:], axis=1)
print("min edge len", min(lengths))

In [ ]:
def barrier_to_color(values):
    return torch.log(values + 1e-5)

In [ ]:
eps = 1
r = 1
alpha = 2
n_samples = 30
n_edges = v.shape[0] - 1

samples = torch.zeros((n_edges * n_samples, 2))
uv = torch.linspace(0, 1, n_samples)
values = torch.zeros((n_edges * n_samples,))
for i in range(n_edges):
    samples[i*n_samples:(i+1)*n_samples,:] = torch.tensor(v[i,:]).view(1, -1) + torch.outer(uv, torch.tensor(v[i+1,:]) - torch.tensor(v[i,:]).view(-1))
    values[i*n_samples:(i+1)*n_samples] = point_edge_potential_exact_pointwise(p, torch.tensor(v[i,:]), torch.tensor(v[i+1,:]), eps, r, alpha, uv)

sample_slider = widgets.IntSlider(
    value=n_samples,
    min=5,
    max=1e6,
    step=1,
    description='n samples:',
    continuous_update=False,
    readout_format='.0f'
)
r_slider = widgets.FloatSlider(
    value=r,
    min=1,
    max=5,
    step=1e-1,
    description='barrier p:',
    continuous_update=False,
    readout_format='.2e'
)
alpha_slider = widgets.FloatSlider(
    value=alpha,
    min=0,
    max=10,
    step=1e-2,
    description='alpha:',
    continuous_update=False,
    readout_format='.2e'
)
eps_slider = widgets.FloatSlider(
    value=eps,
    min=0,
    max=5,
    step=1e-3,
    description='eps:',
    continuous_update=False,
    readout_format='.3e'
)
X_slider = widgets.FloatSlider(
    value=p[0],
    min=-5,
    max=5,
    step=1e-3,
    description='X:',
    continuous_update=False,
    readout_format='.3e'
)
Y_slider = widgets.FloatSlider(
    value=p[1],
    min=-5,
    max=5,
    step=1e-3,
    description='Y:',
    continuous_update=False,
    readout_format='.3e'
)

sliders = [r_slider,alpha_slider,eps_slider,X_slider,Y_slider,sample_slider]
container = widgets.VBox([
    widgets.HBox([r_slider,alpha_slider,eps_slider]),
    widgets.HBox([X_slider,Y_slider])])

trace1 = go.Scatter(x=samples[:,0].detach().numpy(), y=samples[:,1].detach().numpy(),
    mode='markers',
    marker=dict(
        size=3,
        color=barrier_to_color(values).detach().numpy(),
        colorscale='Viridis',  # Choose a colorscale
        colorbar=dict(title='log(potential)')
    ), name="")
g = go.FigureWidget(data=[trace1],
                    layout=go.Layout(width=800, height=600))

def response(change):
    values = torch.zeros((n_edges * n_samples,))
    for i in range(n_edges):
        values[i*n_samples:(i+1)*n_samples] = point_edge_potential_exact_pointwise(torch.tensor([X_slider.value, Y_slider.value]), torch.tensor(v[i,:]), torch.tensor(v[i+1,:]), eps_slider.value, r_slider.value, alpha_slider.value, uv)

    with g.batch_update():
        g.data[0].marker["color"] = barrier_to_color(values).detach().numpy()

for s in sliders:
    s.observe(response, names="value")

widgets.VBox([container,
              g])